In [51]:
import pymysql.cursors
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import RFE
from sklearn.svm import SVR
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression


In [3]:
connection = pymysql.connect(host='localhost',
        user='DL',
        password='121418',
        db='tip',
        charset='utf8',
        cursorclass=pymysql.cursors.DictCursor)

In [4]:
conn = pymysql.connect(host='localhost', user='root', 
                       password='1012', db='tip', charset='utf8',
                       autocommit=True, cursorclass=pymysql.cursors.DictCursor)
try:

    with conn.cursor() as curs:
      sql = "SELECT * FROM tips;"
      curs.execute(sql)
      rs = curs.fetchall()

      # DB에서 받아온 값을 DataFrame에 넣음

      df = pd.DataFrame(rs)
      print(df)
      #df.to_csv('query.csv')

finally:

   conn.close()

     total_bill   tip     sex smoker   day    time  size
0         16.99  1.01  Female     No   Sun  Dinner     2
1         10.34  1.66    Male     No   Sun  Dinner     3
2         21.01  3.50    Male     No   Sun  Dinner     3
3         23.68  3.31    Male     No   Sun  Dinner     2
4         24.59  3.61  Female     No   Sun  Dinner     4
..          ...   ...     ...    ...   ...     ...   ...
239       29.03  5.92    Male     No   Sat  Dinner     3
240       27.18  2.00  Female    Yes   Sat  Dinner     2
241       22.67  2.00    Male    Yes   Sat  Dinner     2
242       17.82  1.75    Male     No   Sat  Dinner     2
243       18.78  3.00  Female     No  Thur  Dinner     2

[244 rows x 7 columns]


In [5]:
df.isnull().sum()

total_bill    0
tip           0
sex           0
smoker        0
day           0
time          0
size          0
dtype: int64

In [6]:
df['sex'].replace({'Female':0, 'Male':1}, inplace=True)

In [7]:
df["smoker"].replace({"No" : 0, "Yes" : 1}, inplace=True)

In [8]:
df["day"].replace({"Thur" : 0, "Fri" : 1, "Sat" : 2, "Sun" : 3}, inplace=True)

In [9]:
df["time"].replace({"Lunch" : 0, "Dinner" : 1}, inplace=True)

In [11]:
df.describe()

,total_bill,tip,sex,smoker,day,time,size
count,244.000000,244.000000,244.000000,244.000000,244.000000,244.000000,244.000000
mean,19.785943,2.998279,0.643443,0.381148,1.725410,0.721311,2.569672
std,8.902412,1.383638,0.479967,0.486667,1.155774,0.449276,0.951100
min,3.070000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,13.347500,2.000000,0.000000,0.000000,0.000000,0.000000,2.000000
50%,17.795000,2.900000,1.000000,0.000000,2.000000,1.000000,2.000000
75%,24.127500,3.562500,1.000000,1.000000,3.000000,1.000000,3.000000
max,50.810000,10.000000,1.000000,1.000000,3.000000,1.000000,6.000000


In [12]:
type(df)

pandas.core.frame.DataFrame

In [14]:
data = df.values

In [16]:
y=df['tip']

In [34]:
X=df.drop('tip',axis=1)

In [35]:
X.describe()

,total_bill,sex,smoker,day,time,size
count,244.000000,244.000000,244.000000,244.000000,244.000000,244.000000
mean,19.785943,0.643443,0.381148,1.725410,0.721311,2.569672
std,8.902412,0.479967,0.486667,1.155774,0.449276,0.951100
min,3.070000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,13.347500,0.000000,0.000000,0.000000,0.000000,2.000000
50%,17.795000,1.000000,0.000000,2.000000,1.000000,2.000000
75%,24.127500,1.000000,1.000000,3.000000,1.000000,3.000000
max,50.810000,1.000000,1.000000,3.000000,1.000000,6.000000


In [36]:
y.describe()

count    244.000000
mean       2.998279
std        1.383638
min        1.000000
25%        2.000000
50%        2.900000
75%        3.562500
max       10.000000
Name: tip, dtype: float64

In [37]:
data=X.values

In [38]:

estimator = SVR(kernel="linear")

In [39]:
rfe = RFE(estimator, n_features_to_select=4)

In [40]:
selector=rfe.fit(X, y)

In [41]:

selector.support_

array([False,  True, False,  True,  True,  True])

In [42]:
df.shape

(244, 7)

In [43]:
for i in range(df.shape[1]):
  print('Column: %d,  Rank: %d' % (i,  rfe.ranking_[i]))

Column: 0,  Rank: 2
Column: 1,  Rank: 1
Column: 2,  Rank: 3
Column: 3,  Rank: 1
Column: 4,  Rank: 1
Column: 5,  Rank: 1


IndexError: index 6 is out of bounds for axis 0 with size 6

In [44]:
df1=pd.DataFrame(X)

In [46]:
df1

,total_bill,sex,smoker,day,time,size
0,16.99,0,0,3,1,2
1,10.34,1,0,3,1,3
2,21.01,1,0,3,1,3
3,23.68,1,0,3,1,2
4,24.59,0,0,3,1,4
...,...,...,...,...,...,...
239,29.03,1,0,2,1,3
240,27.18,0,1,2,1,2
241,22.67,1,1,2,1,2
242,17.82,1,0,2,1,2


In [45]:
df1.corr()

,total_bill,sex,smoker,day,time,size
total_bill,1.000000,0.144877,0.085721,0.173693,0.183118,0.598315
sex,0.144877,1.000000,0.002816,0.230791,0.205231,0.086195
smoker,0.085721,0.002816,1.000000,-0.032653,0.054921,-0.133178
day,0.173693,0.230791,-0.032653,1.000000,0.874366,0.165350
time,0.183118,0.205231,0.054921,0.874366,1.000000,0.103411
size,0.598315,0.086195,-0.133178,0.165350,0.103411,1.000000


In [48]:
trans = PCA(n_components=4)

X_dim = trans.fit_transform(df1)

df2 = pd.DataFrame(X_dim)

In [49]:
df2

,0,1,2,3
0,-2.800690,-1.275890,-0.370382,-0.025780
1,-9.362519,-1.640035,0.967378,-0.220626
2,1.281697,-1.322107,0.335671,-0.370441
3,3.880995,-1.172776,-0.790011,-0.602359
4,4.909445,-1.188986,1.114881,0.256366
...,...,...,...,...
239,9.259332,-0.145865,-0.086833,-0.513686
240,7.346314,-0.011672,-1.149002,0.650726
241,2.855064,-0.242279,-0.905546,0.231404
242,-1.987829,-0.410109,-0.390762,-0.550717


In [50]:
df2.corr()

,0,1,2,3
0,1.000000e+00,-5.450401e-17,1.065233e-17,2.967462e-17
1,-5.450401e-17,1.000000e+00,-3.035063e-17,-2.590366e-16
2,1.065233e-17,-3.035063e-17,1.000000e+00,-1.384733e-16
3,2.967462e-17,-2.590366e-16,-1.384733e-16,1.000000e+00


In [52]:
fs = SelectKBest(score_func=f_regression, k=4)
# apply feature selection
X_selected = fs.fit_transform(df1, y)
print(X_selected.shape)

(244, 4)


In [54]:
df3 = pd.DataFrame(X_selected)

In [55]:
df3

,0,1,2,3
0,16.99,3.0,1.0,2.0
1,10.34,3.0,1.0,3.0
2,21.01,3.0,1.0,3.0
3,23.68,3.0,1.0,2.0
4,24.59,3.0,1.0,4.0
...,...,...,...,...
239,29.03,2.0,1.0,3.0
240,27.18,2.0,1.0,2.0
241,22.67,2.0,1.0,2.0
242,17.82,2.0,1.0,2.0


In [56]:
df3.corr()

,0,1,2,3
0,1.000000,0.173693,0.183118,0.598315
1,0.173693,1.000000,0.874366,0.165350
2,0.183118,0.874366,1.000000,0.103411
3,0.598315,0.165350,0.103411,1.000000
